In [ ]:
# ==========================================================
# Phase 3 – Ensemble Learning (Stacking Regressor)
# ==========================================================

from sklearn.ensemble import StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt
import pandas as pd
import numpy as np
import joblib

# ---------------------------------------
# Rule-based feature engineering
# ---------------------------------------

def add_rule_based_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add rule-based / threshold features on top of Phase 2 engineered features.

    Assumes df has:
      - trip_duration_days
      - miles_traveled
      - total_receipts_amount
      - cost_per_day
      - cost_per_mile
      - miles_per_day
      - cost_ratio
    """
    df = df.copy()

    # --- Trip duration buckets ---
    df["is_short_trip"]  = (df["trip_duration_days"] <= 3).astype(int)
    df["is_medium_trip"] = df["trip_duration_days"].between(4, 7).astype(int)
    df["is_long_trip"]   = (df["trip_duration_days"] >= 8).astype(int)

    # --- Mileage buckets ---
    df["is_local_miles"]    = (df["miles_traveled"] <= 50).astype(int)
    df["is_regional_miles"] = df["miles_traveled"].between(51, 300).astype(int)
    df["is_long_miles"]     = df["miles_traveled"].between(301, 800).astype(int)
    df["is_very_long_miles"]= (df["miles_traveled"] > 800).astype(int)

    # --- Receipt amount buckets ---
    df["receipts_low"]    = (df["total_receipts_amount"] <= 100).astype(int)
    df["receipts_mid"]    = df["total_receipts_amount"].between(100, 500).astype(int)
    df["receipts_high"]   = df["total_receipts_amount"].between(500, 1500).astype(int)
    df["receipts_extreme"]= (df["total_receipts_amount"] > 1500).astype(int)

    # --- Cost-per-mile thresholds ---
    df["cpm_very_low"] = (df["cost_per_mile"] < 0.5).astype(int)
    df["cpm_low"]      = df["cost_per_mile"].between(0.5, 1.5).astype(int)
    df["cpm_mid"]      = df["cost_per_mile"].between(1.5, 5.0).astype(int)
    df["cpm_high"]     = (df["cost_per_mile"] > 5.0).astype(int)

    # --- Cost-per-day thresholds ---
    df["cpd_low"]    = (df["cost_per_day"] <= 150).astype(int)
    df["cpd_mid"]    = df["cost_per_day"].between(150, 300).astype(int)
    df["cpd_high"]   = df["cost_per_day"].between(300, 600).astype(int)
    df["cpd_extreme"]= (df["cost_per_day"] > 600).astype(int)

    # --- Miles per day: "mileage-heavy" trips ---
    df["miles_per_day_light"]  = (df["miles_per_day"] <= 50).astype(int)
    df["miles_per_day_normal"] = df["miles_per_day"].between(51, 200).astype(int)
    df["miles_per_day_heavy"]  = (df["miles_per_day"] > 200).astype(int)

    # --- Simple interaction-style flags (candidate rules) ---
    # Short trips with high receipts → maybe special handling
    df["short_high_receipts"] = ((df["trip_duration_days"] <= 3) &
                                 (df["total_receipts_amount"] > 500)).astype(int)

    # Long trips with low miles → possibly hotel-heavy, mileage-light
    df["long_low_miles"] = ((df["trip_duration_days"] >= 8) &
                            (df["miles_traveled"] < 100)).astype(int)

    # High mileage & low receipts → maybe mileage-structured
    df["high_miles_low_receipts"] = ((df["miles_traveled"] > 500) &
                                     (df["total_receipts_amount"] < 300)).astype(int)

    # Ratio-based: receipts per mile/day above a rough threshold
    df["receipt_heavy_per_mile"] = ((df["total_receipts_amount"] /
                                     df["miles_traveled"].replace(0, np.nan)) > 3).fillna(0).astype(int)
    df["receipt_heavy_per_day"]  = ((df["total_receipts_amount"] /
                                     df["trip_duration_days"].replace(0, np.nan)) > 300).fillna(0).astype(int)

    return df


# Load the enhanced dataset from Phase 2
combined_df = pd.read_csv("phase2_features_baseline_models.csv")

print("Dataset loaded for Phase 3!")
print("Shape:", combined_df.shape)

# Add rule-based features on top of Phase 2 engineered features
combined_df = add_rule_based_features(combined_df)
print("Rule-based features added. New shape:", combined_df.shape)


# ---------------------------------------
# Select Features and Target
# ---------------------------------------
features = [
    # original continuous features
    "trip_duration_days",
    "miles_traveled",
    "total_receipts_amount",
    "cost_per_day",
    "cost_per_mile",
    "miles_per_day",
    "cost_ratio",

    # rule-based trip duration flags
    "is_short_trip",
    "is_medium_trip",
    "is_long_trip",

    # rule-based mileage flags
    "is_local_miles",
    "is_regional_miles",
    "is_long_miles",
    "is_very_long_miles",

    # receipts buckets
    "receipts_low",
    "receipts_mid",
    "receipts_high",
    "receipts_extreme",

    # cost per mile buckets
    "cpm_very_low",
    "cpm_low",
    "cpm_mid",
    "cpm_high",

    # cost per day buckets
    "cpd_low",
    "cpd_mid",
    "cpd_high",
    "cpd_extreme",

    # miles per day buckets
    "miles_per_day_light",
    "miles_per_day_normal",
    "miles_per_day_heavy",

    # interaction-style flags
    "short_high_receipts",
    "long_low_miles",
    "high_miles_low_receipts",
    "receipt_heavy_per_mile",
    "receipt_heavy_per_day",
]


target = "reimbursement"

X = combined_df[features]
y = combined_df[target]

# ---------------------------------------
# Manual 75/25 split
# ---------------------------------------
split = int(0.75 * len(combined_df))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print(f"Training Samples: {len(X_train)}")
print(f"Testing Samples:  {len(X_test)}")
print(" Training Ensemble Model...")

# ----------------------------------------------------------
# Define Base Models
# ----------------------------------------------------------
base_models = [
    ('decision_tree', DecisionTreeRegressor(random_state=42)),
    ('random_forest', RandomForestRegressor(n_estimators=200, random_state=42)),
    ('gradient_boosting', GradientBoostingRegressor(random_state=42))
]

# ----------------------------------------------------------
# Meta-Learner (Final model that blends predictions)
# ----------------------------------------------------------
meta_model = LinearRegression()

# ----------------------------------------------------------
# Build Stacking Regressor
# ----------------------------------------------------------
stack_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # includes original features + model predictions
)

# Train Stacking Ensemble
stack_model.fit(X_train, y_train)

# ----------------------------------------------------------
# Evaluate Model
# ----------------------------------------------------------
stack_pred = stack_model.predict(X_test)

mae = mean_absolute_error(y_test, stack_pred)
rmse = sqrt(mean_squared_error(y_test, stack_pred))
r2 = r2_score(y_test, stack_pred)

print("\n Ensemble Model Performance (Stacking Regressor):")
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")

# ----------------------------------------------------------
# Plot Actual vs Predicted
# ----------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(6,5))
sns.scatterplot(x=y_test, y=stack_pred, alpha=0.7, color='teal')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Actual Reimbursement")
plt.ylabel("Predicted Reimbursement (Ensemble)")
plt.title("Actual vs Predicted — Stacking Ensemble")
plt.tight_layout()
plt.show()


#testing the new model
def prediction_match_report(y_true, y_pred):
    """
    Inline prediction match test (no CSV needed).

    - Exact matches: |pred - actual| <= $0.01
    - Close matches: |pred - actual| <= $1.00
    - ±$5 Accuracy:  |pred - actual| <= $5.00
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    abs_diff = np.abs(y_pred - y_true)

    exact = (abs_diff <= 0.01).sum() / len(y_true)
    close = (abs_diff <= 1.00).sum() / len(y_true)
    within_5 = (abs_diff <= 5.00).sum() / len(y_true)

    print("\n=====  Phase 3 Prediction Match Test =====")
    print(f"Exact Match Rate (<= $0.01): {exact:.4f}")
    print(f"Close Match Rate (<= $1.00): {close:.4f}")
    print(f"±$5 Accuracy: {within_5:.4f}")

# Inline prediction match evaluation (no CSV)
prediction_match_report(y_test, stack_pred)

# ----------------------------------------------------------
#  Residual Calibration – Learn Corrections on Top of Ensemble
# ----------------------------------------------------------
from sklearn.ensemble import GradientBoostingRegressor

# 1) Compute base predictions on TRAIN and TEST
train_base_pred = stack_model.predict(X_train)
test_base_pred = stack_pred  # already computed above on X_test

# 2) Fit residual model on training residuals (actual - base_pred)
train_residuals = y_train - train_base_pred

residual_model = GradientBoostingRegressor(
    random_state=42,
    n_estimators=100,
    learning_rate=0.05,   # smaller steps
    max_depth=2,          # shallow trees to avoid overfitting
    min_samples_leaf=20   # smoother corrections
)

residual_model.fit(X_train, train_residuals)

print("\n Residual correction model trained!")

# 3) Get correction predictions for train and test
train_corrections = residual_model.predict(X_train)
test_corrections = residual_model.predict(X_test)

# 4) Calibrated final predictions
train_final_pred = train_base_pred + train_corrections
test_final_pred = test_base_pred + test_corrections

# 5) Evaluate calibrated model
calib_mae = mean_absolute_error(y_test, test_final_pred)
calib_rmse = sqrt(mean_squared_error(y_test, test_final_pred))
calib_r2 = r2_score(y_test, test_final_pred)

print("\n  Calibrated Ensemble Performance (with Residual Model):")
print(f"MAE:  {calib_mae:.4f}")
print(f"RMSE: {calib_rmse:.4f}")
print(f"R²:   {calib_r2:.4f}")

# 6) Match-rate diagnostics for base vs calibrated model
print("\n=== Base Ensemble Match Rates (Test) ===")
prediction_match_report(y_test, test_base_pred)

print("\n=== Calibrated Ensemble Match Rates (Test) ===")
prediction_match_report(y_test, test_final_pred)

# 7) (Optional) Also see how calibration behaves on train set
print("\n=== Calibrated Ensemble Match Rates (Train) ===")
prediction_match_report(y_train, train_final_pred)







In [ ]:
# ==========================================================
# Phase 3 – Ensemble Learning (Stacking Regressor)
# With ONLY Original + PRD Logic Features (NO rule flags)
# ==========================================================

from sklearn.ensemble import StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt
import pandas as pd
import numpy as np
import joblib

# ---------------------------------------
# PRD-driven feature engineering
# ---------------------------------------

def add_prd_logic_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds PRD-derived behavioral features:
      - spend_per_day curved behavior
      - efficiency reward band (4–6 days & 180–220 mi/day)
      - log miles decay
      - receipt cents anomalies (.49 / .99)
      - receipts curve near $700 (bonus)
      - extreme receipt conditions
    Assumes Phase 2 features already exist:
        cost_per_day, miles_traveled, trip_duration_days, total_receipts_amount
    """

    df = df.copy()

    # === Core Derived Behavior ===
    df["spend_per_day"] = df["cost_per_day"]

    df["miles_per_day_safe"] = (
        df["miles_traveled"] / df["trip_duration_days"].clip(lower=1)
    )

    df["log_miles_traveled"] = np.log1p(df["miles_traveled"])

    # === Receipt cents anomalies (.49 / .99 quirk) ===
    df["receipt_cents"] = np.round(
        df["total_receipts_amount"] - np.floor(df["total_receipts_amount"]), 2
    )
    df["receipt_is_point_49_or_99"] = df["receipt_cents"].isin([0.49, 0.99]).astype(int)

    # === Spend per day optimal band (75–120) & penalties ===
    df["spend_per_day_good_band"] = df["spend_per_day"].between(75, 120).astype(int)
    df["spend_per_day_low"] = (df["spend_per_day"] < 50).astype(int)
    df["spend_per_day_high"] = (df["spend_per_day"] > 120).astype(int)

    # === Receipt non-linearity around ~700 ===
    df["receipts_near_700"] = df["total_receipts_amount"].between(600, 800).astype(int)
    df["receipts_very_low"] = (df["total_receipts_amount"] < 50).astype(int)
    df["receipts_very_high"] = (df["total_receipts_amount"] > 1000).astype(int)

    # === Efficiency Sweet Spot (4–6 days ∧ 180–220 mi/day) ===
    duration_mask = df["trip_duration_days"].between(4, 6)
    miles_mask = df["miles_per_day_safe"].between(180, 220)

    df["is_efficiency_sweet_spot"] = (duration_mask & miles_mask).astype(int)
    df["efficiency_score"] = (
        df["miles_per_day_safe"] * df["is_efficiency_sweet_spot"]
    )

    return df

# ----------------------------------------------------------
# Load Phase 2 dataset + add PRD features
# ----------------------------------------------------------
combined_df = pd.read_csv("phase2_features_baseline_models.csv")

print("Dataset loaded for Phase 3!")
print("Shape:", combined_df.shape)

combined_df = add_prd_logic_features(combined_df)
print("PRD-driven logic features added. New shape:", combined_df.shape)

# ---------------------------------------
# Feature Selection (ONLY original + PRD)
# ---------------------------------------
features = [
    # Phase 2 continuous engineered inputs
    "trip_duration_days",
    "miles_traveled",
    "total_receipts_amount",
    "cost_per_day",
    "cost_per_mile",
    "miles_per_day",
    "cost_ratio",

    # PRD logic features
    "spend_per_day",
    "miles_per_day_safe",
    "log_miles_traveled",
    "receipt_cents",
    "receipt_is_point_49_or_99",
    "spend_per_day_good_band",
    "spend_per_day_low",
    "spend_per_day_high",
    "receipts_near_700",
    "receipts_very_low",
    "receipts_very_high",
    "efficiency_score",
    "is_efficiency_sweet_spot",
]

target = "reimbursement"

X = combined_df[features]
y = combined_df[target]

# ---------------------------------------
# Manual 75/25 split
# ---------------------------------------
split = int(0.75 * len(combined_df))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print(f"Training Samples: {len(X_train)}")
print(f"Testing Samples:  {len(X_test)}")
print(" Training Ensemble Model...")

# ----------------------------------------------------------
# Define Base & Stacking (same)
# ----------------------------------------------------------
base_models = [
    ('decision_tree', DecisionTreeRegressor(random_state=42)),
    ('random_forest', RandomForestRegressor(n_estimators=200, random_state=42)),
    ('gradient_boosting', GradientBoostingRegressor(random_state=42))
]

meta_model = LinearRegression()

stack_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True
)

stack_model.fit(X_train, y_train)

# ----------------------------------------------------------
# Evaluate Base Model
# ----------------------------------------------------------
stack_pred = stack_model.predict(X_test)

mae = mean_absolute_error(y_test, stack_pred)
rmse = sqrt(mean_squared_error(y_test, stack_pred))
r2 = r2_score(y_test, stack_pred)

print("\n Ensemble Model Performance (Stacking Regressor):")
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")

# ----------------------------------------------------------
# Prediction Match Test
# ----------------------------------------------------------
def prediction_match_report(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    abs_diff = np.abs(y_pred - y_true)
    print("\n=====  Phase 3 Prediction Match Test =====")
    print(f"Exact Match Rate (<= $0.01): {(abs_diff<=0.01).mean():.4f}")
    print(f"Close Match Rate (<= $1.00): {(abs_diff<=1.00).mean():.4f}")
    print(f"±$5 Accuracy: {(abs_diff<=5.00).mean():.4f}")

prediction_match_report(y_test, stack_pred)

# ----------------------------------------------------------
# Residual Calibration
# ----------------------------------------------------------
from sklearn.ensemble import GradientBoostingRegressor as GBRResidual

train_base_pred = stack_model.predict(X_train)
train_residuals = y_train - train_base_pred

residual_model = GBRResidual(
    random_state=42,
    n_estimators=100,
    learning_rate=0.05,
    max_depth=2,
    min_samples_leaf=20
)

residual_model.fit(X_train, train_residuals)

test_final_pred = stack_pred + residual_model.predict(X_test)

print("\n=== Calibrated Ensemble Match Rates (Test) ===")
prediction_match_report(y_test, test_final_pred)


In [ ]:
# ==========================================================
# Phase 3 – Ensemble Learning (Stacking Regressor + PRD Logic + Residual Calibration)
# ==========================================================

from sklearn.ensemble import StackingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt
import pandas as pd
import numpy as np
import joblib

import matplotlib.pyplot as plt
import seaborn as sns


# ---------------------------------------
# PRD-driven logic feature engineering
# ---------------------------------------

def add_prd_logic_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add PRD-driven behavioral features on top of Phase 2 engineered features.

    Assumes df has:
      - trip_duration_days
      - miles_traveled
      - total_receipts_amount
      - cost_per_day
      - cost_per_mile
      - miles_per_day
      - cost_ratio
    """
    df = df.copy()

    # --- Core derived features ---
    # Spend per day
    df["spend_per_day"] = df["total_receipts_amount"] / df["trip_duration_days"].replace(0, np.nan)

    # Log of miles (for non-linear mileage curve)
    df["log_miles"] = np.log1p(df["miles_traveled"])

    # Cents pattern in receipts (rounding quirks)
    df["receipt_cents"] = ((df["total_receipts_amount"] * 100).round().astype(int) % 100)

    # Efficiency score: active only in sweet-spot duration 4–6 days
    df["efficiency_score"] = df["miles_per_day"] * df["trip_duration_days"].between(4, 6).astype(int)

    # --- Duration effects: bonuses / penalties ---
    # 5-day bonus, >7-day penalty (vacation penalty)
    df["duration_bonus_flag"] = (df["trip_duration_days"] == 5).astype(int)
    df["long_trip_penalty_flag"] = (df["trip_duration_days"] > 7).astype(int)

    # --- Mileage per day sweet spot and penalties ---
    df["sweetspot_miles_per_day"] = df["miles_per_day"].between(180, 220).astype(int)
    df["overshoot_miles_per_day"] = (df["miles_per_day"] > 300).astype(int)

    # --- Spending rate: optimal band and penalties ---
    df["optimal_spend_band"] = df["spend_per_day"].between(75, 120).astype(int)
    df["low_spend_penalty"] = (df["spend_per_day"] < 50).astype(int)
    df["high_spend_penalty"] = (df["spend_per_day"] > 120).astype(int)

    # --- Receipt non-linearity around total receipts ---
    # Approx: best around 600–800; penalties for very low/high
    df["receipts_optimal_band"] = df["total_receipts_amount"].between(600, 800).astype(int)
    df["receipts_outside_band"] = (
        (df["total_receipts_amount"] < 50) | (df["total_receipts_amount"] > 1000)
    ).astype(int)

    return df


# ---------------------------------------
# Load Phase 2 dataset and add PRD features
# ---------------------------------------

combined_df = pd.read_csv("phase2_features_baseline_models.csv")
print("Dataset loaded for Phase 3!")
print("Shape:", combined_df.shape)

combined_df = add_prd_logic_features(combined_df)
print("PRD-driven logic features added. New shape:", combined_df.shape)


# ---------------------------------------
# Select Features and Target
# ---------------------------------------
features = [
    # Original continuous features
    "trip_duration_days",
    "miles_traveled",
    "total_receipts_amount",
    "cost_per_day",
    "cost_per_mile",
    "miles_per_day",
    "cost_ratio",

    # PRD-driven features
    "spend_per_day",
    "log_miles",
    "receipt_cents",
    "efficiency_score",
    "duration_bonus_flag",
    "long_trip_penalty_flag",
    "sweetspot_miles_per_day",
    "overshoot_miles_per_day",
    "optimal_spend_band",
    "low_spend_penalty",
    "high_spend_penalty",
    "receipts_optimal_band",
    "receipts_outside_band",
]

target = "reimbursement"

X = combined_df[features]
y = combined_df[target]


# ---------------------------------------
# Manual 75/25 split (to match earlier phases)
# ---------------------------------------
split = int(0.75 * len(combined_df))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print(f"Training Samples: {len(X_train)}")
print(f"Testing Samples:  {len(X_test)}")
print(" Training Ensemble Model...")


# ---------------------------------------
# Build Stacking Regressor
# ---------------------------------------
base_models = [
    ('decision_tree', DecisionTreeRegressor(
    random_state=42,
    max_depth=5,
    min_samples_leaf=20,
    min_samples_split=40
)),
    ('random_forest', RandomForestRegressor(
    n_estimators=400,        # more trees = smoother average
    random_state=42,
    max_depth=None,         # or try 12–16 if you see overfitting
    min_samples_leaf=5,     # avoid ultra-tiny leaves
    max_features="sqrt",    # default, but good; could try 0.5 too
    n_jobs=-1               # if you want speed-up
)),
    ('gradient_boosting', GradientBoostingRegressor(
    random_state=42,
    n_estimators=250,      # more boosting rounds
    learning_rate=0.03,   # smaller steps
    max_depth=3,          # more complex interactions
    min_samples_leaf=10
))
]

meta_model = LinearRegression()

stack_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True,   # include original features along with base model preds
)

# Train Stacking Ensemble
stack_model.fit(X_train, y_train)

# Base predictions on test
stack_pred = stack_model.predict(X_test)

mae = mean_absolute_error(y_test, stack_pred)
rmse = sqrt(mean_squared_error(y_test, stack_pred))
r2 = r2_score(y_test, stack_pred)

print("\n Ensemble Model Performance (Stacking Regressor):")
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")


# ---------------------------------------
# Plot Actual vs Predicted (base ensemble)
# ---------------------------------------
plt.figure(figsize=(6, 5))
sns.scatterplot(x=y_test, y=stack_pred, alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Actual Reimbursement")
plt.ylabel("Predicted Reimbursement (Ensemble)")
plt.title("Actual vs Predicted — Stacking Ensemble (Base)")
plt.tight_layout()
plt.show()


# ---------------------------------------
# Match-rate helper
# ---------------------------------------
def prediction_match_report(y_true, y_pred, title="Phase 3 Prediction Match Test"):
    """
    Inline prediction match test.

    - Exact matches: |pred - actual| <= $0.01
    - Close matches: |pred - actual| <= $1.00
    - ±$5 Accuracy:  |pred - actual| <= $5.00
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    abs_diff = np.abs(y_pred - y_true)

    exact = (abs_diff <= 0.01).sum() / len(y_true)
    close = (abs_diff <= 1.00).sum() / len(y_true)
    within_5 = (abs_diff <= 5.00).sum() / len(y_true)

    print(f"\n=====  {title} =====")
    print(f"Exact Match Rate (<= $0.01): {exact:.4f}")
    print(f"Close Match Rate (<= $1.00): {close:.4f}")
    print(f"±$5 Accuracy: {within_5:.4f}")


# Base ensemble match rates
prediction_match_report(y_test, stack_pred, title="Base Ensemble Match Rates (Test)")


# ---------------------------------------
# Residual Calibration – Learn Corrections on Top of Ensemble
# ---------------------------------------

# 1) Compute base predictions on TRAIN and TEST
train_base_pred = stack_model.predict(X_train)
test_base_pred = stack_pred  # already computed on X_test

# 2) Compute training residuals
train_residuals = y_train - train_base_pred
abs_res = np.abs(train_residuals)

# 3) Build sample weights to emphasize large residuals
sample_weights = np.ones_like(abs_res, dtype=float)
sample_weights[abs_res > 1.0] *= 3.0   # more weight for errors > $1
sample_weights[abs_res > 5.0] *= 5.0   # even more for errors > $5

# 4) Extended feature space for residual model: [X, base_pred]
X_train_res = X_train.copy()
X_train_res["base_pred"] = train_base_pred

X_test_res = X_test.copy()
X_test_res["base_pred"] = test_base_pred

# 5) Residual model (small gradient boosting regressor)
residual_model = GradientBoostingRegressor(
    random_state=42,
    n_estimators=150,
    learning_rate=0.05,
    max_depth=2,
    min_samples_leaf=10,
)

residual_model.fit(X_train_res, train_residuals, sample_weight=sample_weights)
print("\n Residual correction model trained!")


# 6) Get correction predictions for train and test
train_corrections = residual_model.predict(X_train_res)
test_corrections = residual_model.predict(X_test_res)

# 7) Calibrated final predictions
train_final_pred = train_base_pred + train_corrections
test_final_pred = test_base_pred + test_corrections

# 8) Evaluate calibrated model (raw floats)
calib_mae = mean_absolute_error(y_test, test_final_pred)
calib_rmse = sqrt(mean_squared_error(y_test, test_final_pred))
calib_r2 = r2_score(y_test, test_final_pred)

print("\n  Calibrated Ensemble Performance (with Residual Model):")
print(f"MAE:  {calib_mae:.4f}")
print(f"RMSE: {calib_rmse:.4f}")
print(f"R²:   {calib_r2:.4f}")

# 9) Match-rate diagnostics: base vs calibrated
prediction_match_report(y_test, test_base_pred, title="Base Ensemble Match Rates (Test)")
prediction_match_report(y_test, test_final_pred, title="Calibrated Ensemble Match Rates (Test – Raw)")


# 10) Optional: enforce money-like rounding (2 decimal cents)
test_final_pred_money = np.round(test_final_pred, 2)
prediction_match_report(y_test, test_final_pred_money,
                        title="Calibrated Ensemble Match Rates (Test – Rounded to Cents)")


# 11) (Optional) See how calibration behaves on training set
prediction_match_report(y_train, train_final_pred, title="Calibrated Ensemble Match Rates (Train)")





In [ ]:
# ==========================================================
# Phase 3 – Ensemble Learning (Stacking Regressor)
# With ONLY Original + PRD Logic Features (NO rule flags)
# ==========================================================

from math import sqrt

import joblib
import numpy as np
import pandas as pd

from sklearn.ensemble import (
    StackingRegressor,
    RandomForestRegressor,
    GradientBoostingRegressor,
    VotingRegressor,
)
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# ---------------------------------------
# PRD-driven feature engineering
# ---------------------------------------

def add_prd_logic_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds PRD-derived behavioral features:
      - spend_per_day curved behavior
      - efficiency reward band (4–6 days & 180–220 mi/day)
      - log miles decay
      - receipt cents anomalies (.49 / .99)
      - receipts curve near $700 (bonus)
      - extreme receipt conditions

    Assumes Phase 2 features already exist:
        cost_per_day, miles_traveled, trip_duration_days, total_receipts_amount
    """
    df = df.copy()

    # === Core Derived Behavior ===
    # Per-day spend: in Phase 2, cost_per_day already behaves like spend_per_day
    df["spend_per_day"] = df["cost_per_day"]

    # Miles per day with safe denominator
    df["miles_per_day_safe"] = (
        df["miles_traveled"] / df["trip_duration_days"].clip(lower=1)
    )

    # Log miles for non-linear mileage curve
    df["log_miles_traveled"] = np.log1p(df["miles_traveled"])

    # === Receipt cents anomalies (.49 / .99 quirk) ===
    df["receipt_cents"] = np.round(
        df["total_receipts_amount"] - np.floor(df["total_receipts_amount"]), 2
    )
    df["receipt_is_point_49_or_99"] = df["receipt_cents"].isin([0.49, 0.99]).astype(int)

    # === Spend per day optimal band (75–120) & penalties ===
    df["spend_per_day_good_band"] = df["spend_per_day"].between(75, 120).astype(int)
    df["spend_per_day_low"] = (df["spend_per_day"] < 50).astype(int)
    df["spend_per_day_high"] = (df["spend_per_day"] > 120).astype(int)

    # === Receipt non-linearity around ~700 ===
    df["receipts_near_700"] = df["total_receipts_amount"].between(600, 800).astype(int)
    df["receipts_very_low"] = (df["total_receipts_amount"] < 50).astype(int)
    df["receipts_very_high"] = (df["total_receipts_amount"] > 1000).astype(int)

    # === Efficiency Sweet Spot (4–6 days ∧ 180–220 mi/day) ===
    duration_mask = df["trip_duration_days"].between(4, 6)
    miles_mask = df["miles_per_day_safe"].between(180, 220)

    df["is_efficiency_sweet_spot"] = (duration_mask & miles_mask).astype(int)
    df["efficiency_score"] = (
        df["miles_per_day_safe"] * df["is_efficiency_sweet_spot"]
    )

    return df


# ----------------------------------------------------------
# Load Phase 2 dataset + add PRD features
# ----------------------------------------------------------
combined_df = pd.read_csv("phase2_features_baseline_models.csv")

print("Dataset loaded for Phase 3!")
print("Shape:", combined_df.shape)

combined_df = add_prd_logic_features(combined_df)
print("PRD-driven logic features added. New shape:", combined_df.shape)

# ---------------------------------------
# Feature Selection (ONLY original + PRD)
# ---------------------------------------
features = [
    # Phase 2 continuous engineered inputs
    "trip_duration_days",
    "miles_traveled",
    "total_receipts_amount",
    "cost_per_day",
    "cost_per_mile",
    "miles_per_day",
    "cost_ratio",

    # PRD logic features
    "spend_per_day",
    "miles_per_day_safe",
    "log_miles_traveled",
    "receipt_cents",
    "receipt_is_point_49_or_99",
    "spend_per_day_good_band",
    "spend_per_day_low",
    "spend_per_day_high",
    "receipts_near_700",
    "receipts_very_low",
    "receipts_very_high",
    "efficiency_score",
    "is_efficiency_sweet_spot",
]

target = "reimbursement"

X = combined_df[features]
y = combined_df[target]

# ---------------------------------------
# Manual 75/25 split
# ---------------------------------------
split = int(0.75 * len(combined_df))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print(f"Training Samples: {len(X_train)}")
print(f"Testing Samples:  {len(X_test)}")
print(" Training Ensemble Model...")

# ----------------------------------------------------------
# Define Base & Stacking
# ----------------------------------------------------------
base_models = [
    ("decision_tree", DecisionTreeRegressor(random_state=42)),
    ("random_forest", RandomForestRegressor(n_estimators=200, random_state=42)),
    ("gradient_boosting", GradientBoostingRegressor(random_state=42)),
]

meta_model = LinearRegression()

stack_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True,
)

stack_model.fit(X_train, y_train)

# ----------------------------------------------------------
# Evaluate Base Stacked Model
# ----------------------------------------------------------
stack_pred = stack_model.predict(X_test)

mae = mean_absolute_error(y_test, stack_pred)
rmse = sqrt(mean_squared_error(y_test, stack_pred))
r2 = r2_score(y_test, stack_pred)

print("\n Ensemble Model Performance (Stacking Regressor):")
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")


# ----------------------------------------------------------
# Prediction Match Test helper
# ----------------------------------------------------------
def prediction_match_report(y_true, y_pred, label="Phase 3 Prediction Match Test"):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    abs_diff = np.abs(y_pred - y_true)
    print(f"\n=====  {label} =====")
    print(f"Exact Match Rate (<= $0.01): {(abs_diff <= 0.01).mean():.4f}")
    print(f"Close Match Rate (<= $1.00): {(abs_diff <= 1.00).mean():.4f}")
    print(f"±$5 Accuracy: {(abs_diff <= 5.00).mean():.4f}")


print("\n=====  Base Ensemble Match Rates (Test) =====")
prediction_match_report(y_test, stack_pred, label="Base Ensemble Match Rates (Test)")

# ----------------------------------------------------------
# Residual Calibration
# ----------------------------------------------------------
from sklearn.ensemble import GradientBoostingRegressor as GBRResidual

train_base_pred = stack_model.predict(X_train)
train_residuals = y_train - train_base_pred

residual_model = GBRResidual(
    random_state=42,
    n_estimators=100,
    learning_rate=0.05,
    max_depth=2,
    min_samples_leaf=20,
)

residual_model.fit(X_train, train_residuals)

print("\n Residual correction model trained!")

# Corrections
train_corrections = residual_model.predict(X_train)
test_corrections = residual_model.predict(X_test)

# Calibrated predictions
train_final_pred = train_base_pred + train_corrections
test_final_pred = stack_pred + test_corrections

# Calibrated metrics
calib_mae = mean_absolute_error(y_test, test_final_pred)
calib_rmse = sqrt(mean_squared_error(y_test, test_final_pred))
calib_r2 = r2_score(y_test, test_final_pred)

print("\n  Calibrated Ensemble Performance (with Residual Model):")
print(f"MAE:  {calib_mae:.4f}")
print(f"RMSE: {calib_rmse:.4f}")
print(f"R²:   {calib_r2:.4f}")

print("\n=====  Calibrated Ensemble Match Rates (Test – Raw) =====")
prediction_match_report(y_test, test_final_pred, label="Calibrated Ensemble Match Rates (Test – Raw)")

# Optional: rounded to cents
test_final_pred_rounded = np.round(test_final_pred, 2)
print("\n=====  Calibrated Ensemble Match Rates (Test – Rounded to Cents) =====")
prediction_match_report(
    y_test, test_final_pred_rounded, label="Calibrated Ensemble Match Rates (Test – Rounded to Cents)"
)

print("\n=====  Calibrated Ensemble Match Rates (Train) =====")
prediction_match_report(y_train, train_final_pred, label="Calibrated Ensemble Match Rates (Train)")



# ==========================================================
# Advanced Techniques – SVR, MLP, Voting Ensemble
# ==========================================================

# --- Support Vector Regression (SVR) ---
svr_model = make_pipeline(
    StandardScaler(),
    SVR(kernel="rbf", C=10.0, epsilon=5.0),
)

svr_model.fit(X_train, y_train)
svr_pred = svr_model.predict(X_test)

svr_mae = mean_absolute_error(y_test, svr_pred)
svr_rmse = sqrt(mean_squared_error(y_test, svr_pred))
svr_r2 = r2_score(y_test, svr_pred)

print("\n SVR Performance:")
print(f"MAE:  {svr_mae:.4f}")
print(f"RMSE: {svr_rmse:.4f}")
print(f"R²:   {svr_r2:.4f}")
prediction_match_report(y_test, svr_pred, label="SVR Match Rates (Test)")


# --- Neural Network (MLPRegressor) ---
mlp_model = make_pipeline(
    StandardScaler(),
    MLPRegressor(
        hidden_layer_sizes=(64, 32),
        activation="relu",
        solver="adam",
        alpha=1e-3,
        max_iter=2000,
        random_state=42,
    ),
)

mlp_model.fit(X_train, y_train)
mlp_pred = mlp_model.predict(X_test)

mlp_mae = mean_absolute_error(y_test, mlp_pred)
mlp_rmse = sqrt(mean_squared_error(y_test, mlp_pred))
mlp_r2 = r2_score(y_test, mlp_pred)

print("\n MLP Performance:")
print(f"MAE:  {mlp_mae:.4f}")
print(f"RMSE: {mlp_rmse:.4f}")
print(f"R²:   {mlp_r2:.4f}")
prediction_match_report(y_test, mlp_pred, label="MLP Match Rates (Test)")


# --- Voting Regressor (simple averaging ensemble) ---
voting_reg = VotingRegressor(
    estimators=[
        ("rf", base_models[1][1]),   # RandomForestRegressor
        ("gbr", base_models[2][1]),  # GradientBoostingRegressor
        ("tree", base_models[0][1]), # DecisionTreeRegressor
    ]
)

voting_reg.fit(X_train, y_train)
voting_pred = voting_reg.predict(X_test)

v_mae = mean_absolute_error(y_test, voting_pred)
v_rmse = sqrt(mean_squared_error(y_test, voting_pred))
v_r2 = r2_score(y_test, voting_pred)

print("\n Voting Regressor Performance:")
print(f"MAE:  {v_mae:.4f}")
print(f"RMSE: {v_rmse:.4f}")
print(f"R²:   {v_r2:.4f}")
prediction_match_report(y_test, voting_pred, label="Voting Regressor Match Rates (Test)")


# ==========================================================
# Rule-Based Learning – Surrogate Tree for Interpretability
# ==========================================================

# Train a shallow tree to mimic the calibrated ensemble behavior
surrogate_tree = DecisionTreeRegressor(
    max_depth=4,
    min_samples_leaf=20,
    random_state=42,
)

# Use calibrated train predictions as the "target" for the surrogate
surrogate_tree.fit(X_train, train_final_pred)

tree_rules = export_text(surrogate_tree, feature_names=list(X.columns))

print("\n=== Surrogate Tree Rules (approximate learned logic) ===")
print(tree_rules)


In [ ]:
# ==========================================================
# Phase 3 – Ensemble Learning (Stacking Regressor)
# With ONLY Original + PRD Logic Features (NO rule flags)
# ==========================================================

from math import sqrt

import joblib
import numpy as np
import pandas as pd

from sklearn.ensemble import (
    StackingRegressor,
    RandomForestRegressor,
    GradientBoostingRegressor,
    VotingRegressor,
    RandomForestClassifier,
)
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import NearestNeighbors

# ---------------------------------------
# PRD-driven feature engineering
# ---------------------------------------

def add_prd_logic_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds PRD-derived behavioral features:
      - spend_per_day curved behavior
      - efficiency reward band (4–6 days & 180–220 mi/day)
      - log miles decay
      - receipt cents anomalies (.49 / .99)
      - receipts curve near $700 (bonus)
      - extreme receipt conditions

    Assumes Phase 2 features already exist:
        cost_per_day, miles_traveled, trip_duration_days, total_receipts_amount
    """
    df = df.copy()

    # === Core Derived Behavior ===
    # Per-day spend: in Phase 2, cost_per_day already behaves like spend_per_day
    df["spend_per_day"] = df["cost_per_day"]

    # Miles per day with safe denominator
    df["miles_per_day_safe"] = (
        df["miles_traveled"] / df["trip_duration_days"].clip(lower=1)
    )

    # Log miles for non-linear mileage curve
    df["log_miles_traveled"] = np.log1p(df["miles_traveled"])

    # === Receipt cents anomalies (.49 / .99 quirk) ===
    df["receipt_cents"] = np.round(
        df["total_receipts_amount"] - np.floor(df["total_receipts_amount"]), 2
    )
    df["receipt_is_point_49_or_99"] = df["receipt_cents"].isin([0.49, 0.99]).astype(int)

    # === Spend per day optimal band (75–120) & penalties ===
    df["spend_per_day_good_band"] = df["spend_per_day"].between(75, 120).astype(int)
    df["spend_per_day_low"] = (df["spend_per_day"] < 50).astype(int)
    df["spend_per_day_high"] = (df["spend_per_day"] > 120).astype(int)

    # === Receipt non-linearity around ~700 ===
    df["receipts_near_700"] = df["total_receipts_amount"].between(600, 800).astype(int)
    df["receipts_very_low"] = (df["total_receipts_amount"] < 50).astype(int)
    df["receipts_very_high"] = (df["total_receipts_amount"] > 1000).astype(int)

    # === Efficiency Sweet Spot (4–6 days ∧ 180–220 mi/day) ===
    duration_mask = df["trip_duration_days"].between(4, 6)
    miles_mask = df["miles_per_day_safe"].between(180, 220)

    df["is_efficiency_sweet_spot"] = (duration_mask & miles_mask).astype(int)
    df["efficiency_score"] = (
        df["miles_per_day_safe"] * df["is_efficiency_sweet_spot"]
    )

    return df


# ----------------------------------------------------------
# Load Phase 2 dataset + add PRD features
# ----------------------------------------------------------
combined_df = pd.read_csv("phase2_features_baseline_models.csv")

print("Dataset loaded for Phase 3!")
print("Shape:", combined_df.shape)

combined_df = add_prd_logic_features(combined_df)
print("PRD-driven logic features added. New shape:", combined_df.shape)

# ---------------------------------------
# Feature Selection (ONLY original + PRD)
# ---------------------------------------
features = [
    # Phase 2 continuous engineered inputs
    "trip_duration_days",
    "miles_traveled",
    "total_receipts_amount",
    "cost_per_day",
    "cost_per_mile",
    "miles_per_day",
    "cost_ratio",

    # PRD logic features
    "spend_per_day",
    "miles_per_day_safe",
    "log_miles_traveled",
    "receipt_cents",
    "receipt_is_point_49_or_99",
    "spend_per_day_good_band",
    "spend_per_day_low",
    "spend_per_day_high",
    "receipts_near_700",
    "receipts_very_low",
    "receipts_very_high",
    "efficiency_score",
    "is_efficiency_sweet_spot",
]

target = "reimbursement"

X = combined_df[features]
y = combined_df[target]
# ==========================================================
# Sanity Checks + Random (Shuffled) Train/Test Split
# ==========================================================

from sklearn.model_selection import train_test_split

print("\n===== 🔍 Sanity Checks =====")

# 1) How quantized is reimbursement? (lots of repeated exact values suggests discrete payout bands)
y_rounded = combined_df[target].round(2)
n_unique = y_rounded.nunique()
top_counts = y_rounded.value_counts().head(15)

print(f"Unique reimbursements (rounded to cents): {n_unique}")
print("Most common payouts (top 15):")
print(top_counts.to_string())

# 2) Do identical feature rows map to multiple reimbursements? (hidden inputs/noise)
tmp = combined_df[features + [target]].copy()
tmp[target] = tmp[target].round(2)  # compare at cent-level

# group by X and count unique y values per identical X row
y_variation = tmp.groupby(features)[target].nunique()
multi_y_count = int((y_variation > 1).sum())
max_y_for_same_x = int(y_variation.max())

print(f"Identical X mapping to >1 reimbursement (cent-rounded): {multi_y_count}")
print(f"Max distinct reimbursements for the same X: {max_y_for_same_x}")

if multi_y_count > 0:
    print("⚠️  Warning: Same inputs sometimes produce different outputs.")
    print("    This suggests hidden variables and/or stochastic behavior, which limits match-rate ceiling.")

# ==========================================================
# Random / Shuffled split (recommended vs first-75% / last-25%)
# ==========================================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    shuffle=True
)

print("\n===== ✅ Random Split Applied =====")
print(f"Training Samples: {len(X_train)}")
print(f"Testing Samples:  {len(X_test)}")



# ----------------------------------------------------------
# Define Base & Stacking
# ----------------------------------------------------------
base_models = [
    ("decision_tree", DecisionTreeRegressor(random_state=42)),
    ("random_forest", RandomForestRegressor(n_estimators=200, random_state=42)),
    ("gradient_boosting", GradientBoostingRegressor(random_state=42)),
]

meta_model = LinearRegression()

stack_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True,
)

stack_model.fit(X_train, y_train)

# ----------------------------------------------------------
# Evaluate Base Stacked Model
# ----------------------------------------------------------
stack_pred = stack_model.predict(X_test)

mae = mean_absolute_error(y_test, stack_pred)
rmse = sqrt(mean_squared_error(y_test, stack_pred))
r2 = r2_score(y_test, stack_pred)

print("\n Ensemble Model Performance (Stacking Regressor):")
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")


# ----------------------------------------------------------
# Prediction Match Test helper
# ----------------------------------------------------------
def prediction_match_report(y_true, y_pred, label="Phase 3 Prediction Match Test"):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    abs_diff = np.abs(y_pred - y_true)
    print(f"\n=====  {label} =====")
    print(f"Exact Match Rate (<= $0.01): {(abs_diff <= 0.01).mean():.4f}")
    print(f"Close Match Rate (<= $1.00): {(abs_diff <= 1.00).mean():.4f}")
    print(f"±$5 Accuracy: {(abs_diff <= 5.00).mean():.4f}")


print("\n=====  Base Ensemble Match Rates (Test) =====")
prediction_match_report(y_test, stack_pred, label="Base Ensemble Match Rates (Test)")

# ----------------------------------------------------------
# Residual Calibration
# ----------------------------------------------------------
from sklearn.ensemble import GradientBoostingRegressor as GBRResidual

train_base_pred = stack_model.predict(X_train)
train_residuals = y_train - train_base_pred

residual_model = GBRResidual(
    random_state=42,
    n_estimators=100,
    learning_rate=0.05,
    max_depth=2,
    min_samples_leaf=20,
)

residual_model.fit(X_train, train_residuals)

print("\n Residual correction model trained!")

# Corrections
train_corrections = residual_model.predict(X_train)
test_corrections = residual_model.predict(X_test)

# Calibrated predictions
train_final_pred = train_base_pred + train_corrections
test_final_pred = stack_pred + test_corrections

# Calibrated metrics
calib_mae = mean_absolute_error(y_test, test_final_pred)
calib_rmse = sqrt(mean_squared_error(y_test, test_final_pred))
calib_r2 = r2_score(y_test, test_final_pred)

print("\n  Calibrated Ensemble Performance (with Residual Model):")
print(f"MAE:  {calib_mae:.4f}")
print(f"RMSE: {calib_rmse:.4f}")
print(f"R²:   {calib_r2:.4f}")

print("\n=====  Calibrated Ensemble Match Rates (Test – Raw) =====")
prediction_match_report(y_test, test_final_pred, label="Calibrated Ensemble Match Rates (Test – Raw)")

# Optional: rounded to cents
test_final_pred_rounded = np.round(test_final_pred, 2)
print("\n=====  Calibrated Ensemble Match Rates (Test – Rounded to Cents) =====")
prediction_match_report(
    y_test, test_final_pred_rounded, label="Calibrated Ensemble Match Rates (Test – Rounded to Cents)"
)

print("\n=====  Calibrated Ensemble Match Rates (Train) =====")
prediction_match_report(y_train, train_final_pred, label="Calibrated Ensemble Match Rates (Train)")


# ==========================================================
# Advanced Techniques – SVR, MLP, Voting Ensemble
# ==========================================================

# --- Support Vector Regression (SVR) ---
svr_model = make_pipeline(
    StandardScaler(),
    SVR(kernel="rbf", C=10.0, epsilon=5.0),
)

svr_model.fit(X_train, y_train)
svr_pred = svr_model.predict(X_test)

svr_mae = mean_absolute_error(y_test, svr_pred)
svr_rmse = sqrt(mean_squared_error(y_test, svr_pred))
svr_r2 = r2_score(y_test, svr_pred)

print("\n SVR Performance:")
print(f"MAE:  {svr_mae:.4f}")
print(f"RMSE: {svr_rmse:.4f}")
print(f"R²:   {svr_r2:.4f}")
prediction_match_report(y_test, svr_pred, label="SVR Match Rates (Test)")


# --- Neural Network (MLPRegressor) ---
mlp_model = make_pipeline(
    StandardScaler(),
    MLPRegressor(
        hidden_layer_sizes=(64, 32),
        activation="relu",
        solver="adam",
        alpha=1e-3,
        max_iter=2000,
        random_state=42,
    ),
)

mlp_model.fit(X_train, y_train)
mlp_pred = mlp_model.predict(X_test)

mlp_mae = mean_absolute_error(y_test, mlp_pred)
mlp_rmse = sqrt(mean_squared_error(y_test, mlp_pred))
mlp_r2 = r2_score(y_test, mlp_pred)

print("\n MLP Performance:")
print(f"MAE:  {mlp_mae:.4f}")
print(f"RMSE: {mlp_rmse:.4f}")
print(f"R²:   {mlp_r2:.4f}")
prediction_match_report(y_test, mlp_pred, label="MLP Match Rates (Test)")


# --- Voting Regressor (simple averaging ensemble) ---
voting_reg = VotingRegressor(
    estimators=[
        ("rf", base_models[1][1]),   # RandomForestRegressor
        ("gbr", base_models[2][1]),  # GradientBoostingRegressor
        ("tree", base_models[0][1]), # DecisionTreeRegressor
    ]
)

voting_reg.fit(X_train, y_train)
voting_pred = voting_reg.predict(X_test)

v_mae = mean_absolute_error(y_test, voting_pred)
v_rmse = sqrt(mean_squared_error(y_test, voting_pred))
v_r2 = r2_score(y_test, voting_pred)

print("\n Voting Regressor Performance:")
print(f"MAE:  {v_mae:.4f}")
print(f"RMSE: {v_rmse:.4f}")
print(f"R²:   {v_r2:.4f}")
prediction_match_report(y_test, voting_pred, label="Voting Regressor Match Rates (Test)")


# ==========================================================
# Rule-Based Learning – Surrogate Tree for Interpretability
# ==========================================================

# Train a shallow tree to mimic the calibrated ensemble behavior
surrogate_tree = DecisionTreeRegressor(
    max_depth=4,
    min_samples_leaf=20,
    random_state=42,
)

# Use calibrated train predictions as the "target" for the surrogate
surrogate_tree.fit(X_train, train_final_pred)

tree_rules = export_text(surrogate_tree, feature_names=list(X.columns))

print("\n=== Surrogate Tree Rules (approximate learned logic) ===")
print(tree_rules)


# ==========================================================
# DISCRETE / TIERED MODELING – Classification over Payout Buckets (FIXED for shuffled split)
# ==========================================================

# 1) Discretize reimbursement into buckets (e.g., $5 tiers)
bucket_size = 5.0  # try 2.0 or 1.0 for finer tiers (but may overfit)

# Build bucket labels on the full dataset, then align by index to train/test
y_bucket_all = (combined_df[target] / bucket_size).round().astype(int)

y_bucket_train = y_bucket_all.loc[y_train.index]
y_bucket_test  = y_bucket_all.loc[y_test.index]

# 2) Map from bucket -> representative payout value (median payout in TRAIN for that bucket)
bucket_to_median = {}
for b in np.sort(y_bucket_train.unique()):
    mask = (y_bucket_train == b).values  # aligned to y_train order
    bucket_to_median[int(b)] = float(np.median(y_train.values[mask]))

def buckets_to_payouts(bucket_array):
    """Convert bucket labels to concrete payout values using TRAIN median per bucket."""
    bucket_array = np.asarray(bucket_array, dtype=int)
    out = np.empty_like(bucket_array, dtype=float)
    for i, b in enumerate(bucket_array):
        out[i] = bucket_to_median.get(int(b), float(b) * bucket_size)  # fallback to bucket center
    return out

# 3) Train a classifier over payout buckets
rf_cls = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_leaf=5,
    random_state=42,
)

rf_cls.fit(X_train, y_bucket_train)

# 4) Predict buckets on test + convert to payout values
bucket_pred_test = rf_cls.predict(X_test)
tier_pred_test = buckets_to_payouts(bucket_pred_test)

# 5) Evaluate using same regression + match metrics
tier_mae = mean_absolute_error(y_test, tier_pred_test)
tier_rmse = sqrt(mean_squared_error(y_test, tier_pred_test))
tier_r2 = r2_score(y_test, tier_pred_test)

print(f"\n Tier Classifier (RandomForestClassifier over ${bucket_size:g} buckets) Performance:")
print(f"MAE:  {tier_mae:.4f}")
print(f"RMSE: {tier_rmse:.4f}")
print(f"R²:   {tier_r2:.4f}")
prediction_match_report(y_test, tier_pred_test, label="Tier Classifier Match Rates (Test)")


# ==========================================================
# CASE-BASED REASONING – kNN over Historical Trips
# ==========================================================

# kNN in feature space: find k most similar training trips and use their payouts
k_neighbors = 5
nn = NearestNeighbors(n_neighbors=k_neighbors)
nn.fit(X_train)

distances, indices = nn.kneighbors(X_test)

# indices shape: (n_test, k_neighbors)
neighbor_reimb = y_train.values[indices]  # reimbursements of nearest neighbors

# Several possible aggregation strategies; use median to stay robust
knn_case_pred = np.median(neighbor_reimb, axis=1)

knn_mae = mean_absolute_error(y_test, knn_case_pred)
knn_rmse = sqrt(mean_squared_error(y_test, knn_case_pred))
knn_r2 = r2_score(y_test, knn_case_pred)

print("\n kNN Case-Based Predictor Performance (k=5, median of neighbor reimbursements):")
print(f"MAE:  {knn_mae:.4f}")
print(f"RMSE: {knn_rmse:.4f}")
print(f"R²:   {knn_r2:.4f}")
prediction_match_report(y_test, knn_case_pred, label="kNN Case-Based Match Rates (Test)")





THe combined system is strickly wrose that best model so far.

In [ ]:
# ==========================================================
# HYBRID META-MODEL: combine calibrated ensemble + tier classifier + kNN
# ==========================================================
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestNeighbors

print("\n\n=====  Building Hybrid Meta-Model (Reg + Tier + kNN) =====")

# ---------- 1) Tiered classifier over payout buckets ----------
bucket_size = 5.0  # you can try 2.0 or 1.0 for finer tiers

combined_df["reimb_bucket"] = (combined_df["reimbursement"] / bucket_size).round().astype(int)

y_bucket = combined_df["reimb_bucket"]
y_bucket_train = y_bucket[:split]
y_bucket_test = y_bucket[split:]

# Map: bucket -> median reimbursement in that bucket (using TRAIN data only)
bucket_to_median = {}
for b in np.unique(y_bucket_train):
    mask = (y_bucket_train == b)
    bucket_to_median[b] = float(np.median(y_train[mask]))

def buckets_to_payouts(bucket_array):
    vals = []
    for b in bucket_array:
        if b in bucket_to_median:
            vals.append(bucket_to_median[b])
        else:
            vals.append(b * bucket_size)
    return np.array(vals)

rf_cls = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_leaf=5,
    random_state=42,
)

rf_cls.fit(X_train, y_bucket_train)

bucket_pred_train = rf_cls.predict(X_train)
bucket_pred_test = rf_cls.predict(X_test)

tier_pred_train = buckets_to_payouts(bucket_pred_train)
tier_pred_test  = buckets_to_payouts(bucket_pred_test)

print("\n Tier classifier trained.")

# ---------- 2) kNN case-based prediction (train + test) ----------

k_neighbors = 5

# For TEST: neighbors from training set (already what we want)
nn_test = NearestNeighbors(n_neighbors=k_neighbors)
nn_test.fit(X_train)
dist_test, idx_test = nn_test.kneighbors(X_test)
neighbor_reimb_test = y_train.values[idx_test]
knn_case_pred_test = np.median(neighbor_reimb_test, axis=1)

# For TRAIN: use k+1 neighbors and drop self to avoid trivial zero distance
nn_train = NearestNeighbors(n_neighbors=k_neighbors + 1)
nn_train.fit(X_train)
dist_train, idx_train = nn_train.kneighbors(X_train)
# Drop the first column (self neighbor)
neighbor_reimb_train = y_train.values[idx_train[:, 1:]]
knn_case_pred_train = np.median(neighbor_reimb_train, axis=1)

print(" kNN case-based predictor trained.")

# ---------- 3) Build meta-features from three models ----------
# Base calibrated ensemble outputs: train_final_pred and test_final_pred
# (already computed earlier in your script)

meta_X_train = np.column_stack([
    train_final_pred,   # calibrated stack regressor
    tier_pred_train,    # tier classifier regression
    knn_case_pred_train # kNN case-based regression
])
meta_y_train = y_train.values

meta_X_test = np.column_stack([
    test_final_pred,
    tier_pred_test,
    knn_case_pred_test
])

# ---------- 4) Meta-learner: linear blend of the three ----------
meta_blender = LinearRegression()
meta_blender.fit(meta_X_train, meta_y_train)

hybrid_pred_test = meta_blender.predict(meta_X_test)
hybrid_pred_test_rounded = np.round(hybrid_pred_test, 2)

hybrid_mae = mean_absolute_error(y_test, hybrid_pred_test)
hybrid_rmse = sqrt(mean_squared_error(y_test, hybrid_pred_test))
hybrid_r2 = r2_score(y_test, hybrid_pred_test)

print("\n Hybrid Meta-Model Performance (Reg + Tier + kNN):")
print(f"MAE:  {hybrid_mae:.4f}")
print(f"RMSE: {hybrid_rmse:.4f}")
print(f"R²:   {hybrid_r2:.4f}")

print("\n=====  Hybrid Meta-Model Match Rates (Test – Raw) =====")
prediction_match_report(y_test, hybrid_pred_test, label="Hybrid Meta-Model (Raw)")

print("\n=====  Hybrid Meta-Model Match Rates (Test – Rounded to Cents) =====")
prediction_match_report(y_test, hybrid_pred_test_rounded, label="Hybrid Meta-Model (Rounded)")


In [ ]:
# ==========================================================
# Phase 3 – Ensemble Learning (Stacking Regressor)
# With ONLY Original + PRD Logic Features (NO rule flags)
# ==========================================================

from math import sqrt

import joblib
import numpy as np
import pandas as pd

from sklearn.ensemble import (
    StackingRegressor,
    RandomForestRegressor,
    GradientBoostingRegressor,
    VotingRegressor,
    RandomForestClassifier,
)
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import NearestNeighbors

# ---------------------------------------
# PRD-driven feature engineering
# ---------------------------------------

def add_prd_logic_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds PRD-derived behavioral features:
      - spend_per_day curved behavior
      - efficiency reward band (4–6 days & 180–220 mi/day)
      - log miles decay
      - receipt cents anomalies (.49 / .99)
      - receipts curve near $700 (bonus)
      - extreme receipt conditions

    Assumes Phase 2 features already exist:
        cost_per_day, miles_traveled, trip_duration_days, total_receipts_amount
    """
    df = df.copy()

    # === Core Derived Behavior ===
    # Per-day spend: in Phase 2, cost_per_day already behaves like spend_per_day
    df["spend_per_day"] = df["cost_per_day"]

    # Miles per day with safe denominator
    df["miles_per_day_safe"] = (
        df["miles_traveled"] / df["trip_duration_days"].clip(lower=1)
    )

    # Log miles for non-linear mileage curve
    df["log_miles_traveled"] = np.log1p(df["miles_traveled"])

    # === Receipt cents anomalies (.49 / .99 quirk) ===
    df["receipt_cents"] = np.round(
        df["total_receipts_amount"] - np.floor(df["total_receipts_amount"]), 2
    )
    df["receipt_is_point_49_or_99"] = df["receipt_cents"].isin([0.49, 0.99]).astype(int)

    # === Spend per day optimal band (75–120) & penalties ===
    df["spend_per_day_good_band"] = df["spend_per_day"].between(75, 120).astype(int)
    df["spend_per_day_low"] = (df["spend_per_day"] < 50).astype(int)
    df["spend_per_day_high"] = (df["spend_per_day"] > 120).astype(int)

    # === Receipt non-linearity around ~700 ===
    df["receipts_near_700"] = df["total_receipts_amount"].between(600, 800).astype(int)
    df["receipts_very_low"] = (df["total_receipts_amount"] < 50).astype(int)
    df["receipts_very_high"] = (df["total_receipts_amount"] > 1000).astype(int)

    # === Efficiency Sweet Spot (4–6 days ∧ 180–220 mi/day) ===
    duration_mask = df["trip_duration_days"].between(4, 6)
    miles_mask = df["miles_per_day_safe"].between(180, 220)

    df["is_efficiency_sweet_spot"] = (duration_mask & miles_mask).astype(int)
    df["efficiency_score"] = (
        df["miles_per_day_safe"] * df["is_efficiency_sweet_spot"]
    )

    return df


# ----------------------------------------------------------
# Load Phase 2 dataset + add PRD features
# ----------------------------------------------------------
combined_df = pd.read_csv("phase2_features_baseline_models.csv")

print("Dataset loaded for Phase 3!")
print("Shape:", combined_df.shape)

combined_df = add_prd_logic_features(combined_df)
print("PRD-driven logic features added. New shape:", combined_df.shape)

# ---------------------------------------
# Feature Selection (ONLY original + PRD)
# ---------------------------------------
features = [
    # Phase 2 continuous engineered inputs
    "trip_duration_days",
    "miles_traveled",
    "total_receipts_amount",
    "cost_per_day",
    "cost_per_mile",
    "miles_per_day",
    "cost_ratio",

    # PRD logic features
    "spend_per_day",
    "miles_per_day_safe",
    "log_miles_traveled",
    "receipt_cents",
    "receipt_is_point_49_or_99",
    "spend_per_day_good_band",
    "spend_per_day_low",
    "spend_per_day_high",
    "receipts_near_700",
    "receipts_very_low",
    "receipts_very_high",
    "efficiency_score",
    "is_efficiency_sweet_spot",
]

target = "reimbursement"

X = combined_df[features]
y = combined_df[target]
# ==========================================================
# Sanity Checks + Random (Shuffled) Train/Test Split
# ==========================================================

from sklearn.model_selection import train_test_split

print("\n=====  Sanity Checks =====")

# 1) How quantized is reimbursement? (lots of repeated exact values suggests discrete payout bands)
y_rounded = combined_df[target].round(2)
n_unique = y_rounded.nunique()
top_counts = y_rounded.value_counts().head(15)

print(f"Unique reimbursements (rounded to cents): {n_unique}")
print("Most common payouts (top 15):")
print(top_counts.to_string())

# 2) Do identical feature rows map to multiple reimbursements? (hidden inputs/noise)
tmp = combined_df[features + [target]].copy()
tmp[target] = tmp[target].round(2)  # compare at cent-level

# group by X and count unique y values per identical X row
y_variation = tmp.groupby(features)[target].nunique()
multi_y_count = int((y_variation > 1).sum())
max_y_for_same_x = int(y_variation.max())

print(f"Identical X mapping to >1 reimbursement (cent-rounded): {multi_y_count}")
print(f"Max distinct reimbursements for the same X: {max_y_for_same_x}")

if multi_y_count > 0:
    print("  Warning: Same inputs sometimes produce different outputs.")
    print("    This suggests hidden variables and/or stochastic behavior, which limits match-rate ceiling.")

# ==========================================================
# Random / Shuffled split (recommended vs first-75% / last-25%)
# ==========================================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    shuffle=True
)

print("\n=====  Random Split Applied =====")
print(f"Training Samples: {len(X_train)}")
print(f"Testing Samples:  {len(X_test)}")



# ----------------------------------------------------------
# Define Base & Stacking
# ----------------------------------------------------------
base_models = [
    ("decision_tree", DecisionTreeRegressor(random_state=42)),
    ("random_forest", RandomForestRegressor(n_estimators=200, random_state=42)),
    ("gradient_boosting", GradientBoostingRegressor(random_state=42)),
]

meta_model = LinearRegression()

stack_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True,
)

stack_model.fit(X_train, y_train)

# ----------------------------------------------------------
# Evaluate Base Stacked Model
# ----------------------------------------------------------
stack_pred = stack_model.predict(X_test)

mae = mean_absolute_error(y_test, stack_pred)
rmse = sqrt(mean_squared_error(y_test, stack_pred))
r2 = r2_score(y_test, stack_pred)

print("\n Ensemble Model Performance (Stacking Regressor):")
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")


# ----------------------------------------------------------
# Prediction Match Test helper
# ----------------------------------------------------------
def prediction_match_report(y_true, y_pred, label="Phase 3 Prediction Match Test"):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    abs_diff = np.abs(y_pred - y_true)
    print(f"\n=====  {label} =====")
    print(f"Exact Match Rate (<= $0.01): {(abs_diff <= 0.01).mean():.4f}")
    print(f"Close Match Rate (<= $1.00): {(abs_diff <= 1.00).mean():.4f}")
    print(f"±$5 Accuracy: {(abs_diff <= 5.00).mean():.4f}")


print("\n=====  Base Ensemble Match Rates (Test) =====")
prediction_match_report(y_test, stack_pred, label="Base Ensemble Match Rates (Test)")

# ----------------------------------------------------------
# Residual Calibration
# ----------------------------------------------------------
from sklearn.ensemble import GradientBoostingRegressor as GBRResidual

train_base_pred = stack_model.predict(X_train)
train_residuals = y_train - train_base_pred

residual_model = GBRResidual(
    random_state=42,
    n_estimators=100,
    learning_rate=0.05,
    max_depth=2,
    min_samples_leaf=20,
)

residual_model.fit(X_train, train_residuals)

print("\n Residual correction model trained!")

# Corrections
train_corrections = residual_model.predict(X_train)
test_corrections = residual_model.predict(X_test)

# Calibrated predictions
train_final_pred = train_base_pred + train_corrections
test_final_pred = stack_pred + test_corrections

# Calibrated metrics
calib_mae = mean_absolute_error(y_test, test_final_pred)
calib_rmse = sqrt(mean_squared_error(y_test, test_final_pred))
calib_r2 = r2_score(y_test, test_final_pred)

print("\n  Calibrated Ensemble Performance (with Residual Model):")
print(f"MAE:  {calib_mae:.4f}")
print(f"RMSE: {calib_rmse:.4f}")
print(f"R²:   {calib_r2:.4f}")

print("\n=====  Calibrated Ensemble Match Rates (Test – Raw) =====")
prediction_match_report(y_test, test_final_pred, label="Calibrated Ensemble Match Rates (Test – Raw)")

# Optional: rounded to cents
test_final_pred_rounded = np.round(test_final_pred, 2)
print("\n=====  Calibrated Ensemble Match Rates (Test – Rounded to Cents) =====")
prediction_match_report(
    y_test, test_final_pred_rounded, label="Calibrated Ensemble Match Rates (Test – Rounded to Cents)"
)

print("\n=====  Calibrated Ensemble Match Rates (Train) =====")
prediction_match_report(y_train, train_final_pred, label="Calibrated Ensemble Match Rates (Train)")


# ==========================================================
# Advanced Techniques – SVR, MLP, Voting Ensemble
# ==========================================================

# --- Support Vector Regression (SVR) ---
svr_model = make_pipeline(
    StandardScaler(),
    SVR(kernel="rbf", C=10.0, epsilon=5.0),
)

svr_model.fit(X_train, y_train)
svr_pred = svr_model.predict(X_test)

svr_mae = mean_absolute_error(y_test, svr_pred)
svr_rmse = sqrt(mean_squared_error(y_test, svr_pred))
svr_r2 = r2_score(y_test, svr_pred)

print("\n SVR Performance:")
print(f"MAE:  {svr_mae:.4f}")
print(f"RMSE: {svr_rmse:.4f}")
print(f"R²:   {svr_r2:.4f}")
prediction_match_report(y_test, svr_pred, label="SVR Match Rates (Test)")


# --- Neural Network (MLPRegressor) ---
mlp_model = make_pipeline(
    StandardScaler(),
    MLPRegressor(
        hidden_layer_sizes=(64, 32),
        activation="relu",
        solver="adam",
        alpha=1e-3,
        max_iter=2000,
        random_state=42,
    ),
)

mlp_model.fit(X_train, y_train)
mlp_pred = mlp_model.predict(X_test)

mlp_mae = mean_absolute_error(y_test, mlp_pred)
mlp_rmse = sqrt(mean_squared_error(y_test, mlp_pred))
mlp_r2 = r2_score(y_test, mlp_pred)

print("\n MLP Performance:")
print(f"MAE:  {mlp_mae:.4f}")
print(f"RMSE: {mlp_rmse:.4f}")
print(f"R²:   {mlp_r2:.4f}")
prediction_match_report(y_test, mlp_pred, label="MLP Match Rates (Test)")


# --- Voting Regressor (simple averaging ensemble) ---
voting_reg = VotingRegressor(
    estimators=[
        ("rf", base_models[1][1]),   # RandomForestRegressor
        ("gbr", base_models[2][1]),  # GradientBoostingRegressor
        ("tree", base_models[0][1]), # DecisionTreeRegressor
    ]
)

voting_reg.fit(X_train, y_train)
voting_pred = voting_reg.predict(X_test)

v_mae = mean_absolute_error(y_test, voting_pred)
v_rmse = sqrt(mean_squared_error(y_test, voting_pred))
v_r2 = r2_score(y_test, voting_pred)

print("\n Voting Regressor Performance:")
print(f"MAE:  {v_mae:.4f}")
print(f"RMSE: {v_rmse:.4f}")
print(f"R²:   {v_r2:.4f}")
prediction_match_report(y_test, voting_pred, label="Voting Regressor Match Rates (Test)")


# ==========================================================
# Rule-Based Learning – Surrogate Tree for Interpretability
# ==========================================================

# Train a shallow tree to mimic the calibrated ensemble behavior
surrogate_tree = DecisionTreeRegressor(
    max_depth=4,
    min_samples_leaf=20,
    random_state=42,
)

# Use calibrated train predictions as the "target" for the surrogate
surrogate_tree.fit(X_train, train_final_pred)

tree_rules = export_text(surrogate_tree, feature_names=list(X.columns))

print("\n=== Surrogate Tree Rules (approximate learned logic) ===")
print(tree_rules)


# ==========================================================
# DISCRETE / TIERED MODELING – Classification over Payout Buckets
# (FIXED for random train_test_split indexing)
# ==========================================================

from sklearn.neighbors import KNeighborsRegressor

bucket_size = 5.0  # try 5.0, 2.0, 1.0 (but 1.0 creates many classes)

# Build bucket labels from y (NOT from sequential slicing)
y_bucket_all = (y.round(2) / bucket_size).round().astype(int)

# Align buckets with the randomized split via indices
y_bucket_train = y_bucket_all.loc[X_train.index]
y_bucket_test  = y_bucket_all.loc[X_test.index]

# Map: bucket -> representative payout (median payout in TRAIN for that bucket)
train_bucket_df = pd.DataFrame({
    "bucket": y_bucket_train,
    "payout": y_train
})
bucket_to_median = train_bucket_df.groupby("bucket")["payout"].median().to_dict()

def buckets_to_payouts(bucket_array):
    bucket_array = np.asarray(bucket_array)
    return np.array([bucket_to_median.get(int(b), float(b) * bucket_size) for b in bucket_array], dtype=float)

rf_cls = RandomForestClassifier(
    n_estimators=400,
    max_depth=None,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)
rf_cls.fit(X_train, y_bucket_train)

bucket_pred_test = rf_cls.predict(X_test)
tier_pred_test = buckets_to_payouts(bucket_pred_test)

print("\n Tier Classifier (RF over buckets) Performance:")
print(f"MAE:  {mean_absolute_error(y_test, tier_pred_test):.4f}")
print(f"RMSE: {sqrt(mean_squared_error(y_test, tier_pred_test)):.4f}")
print(f"R²:   {r2_score(y_test, tier_pred_test):.4f}")
prediction_match_report(y_test, tier_pred_test, label="Tier Classifier Match Rates (Test)")


# ==========================================================
# CASE-BASED REASONING – kNN Regressor (IMPORTANT: scaling + distance weights)
# ==========================================================

# Your previous NearestNeighbors version used raw feature scales
# => distances dominated by receipts/miles. Scale helps a lot for neighbor quality.

knn_reg = make_pipeline(
    StandardScaler(),
    KNeighborsRegressor(
        n_neighbors=3,         # try 1,3,5,7
        weights="distance",    # closer neighbors matter more
        metric="minkowski",
        p=2
    )
)

knn_reg.fit(X_train, y_train)
knn_pred = knn_reg.predict(X_test)

print("\n kNN Regressor Performance (scaled, distance-weighted):")
print(f"MAE:  {mean_absolute_error(y_test, knn_pred):.4f}")
print(f"RMSE: {sqrt(mean_squared_error(y_test, knn_pred)):.4f}")
print(f"R²:   {r2_score(y_test, knn_pred):.4f}")
prediction_match_report(y_test, knn_pred, label="kNN Regressor Match Rates (Test)")





Directly optimize close match rate




In [ ]:
# ==========================================================
# Phase 3 – Ensemble Learning (Stacking Regressor)
# With ONLY Original + PRD Logic Features (NO rule flags)
# ==========================================================

from math import sqrt

import joblib
import numpy as np
import pandas as pd

from sklearn.ensemble import (
    StackingRegressor,
    RandomForestRegressor,
    GradientBoostingRegressor,
    VotingRegressor,
    RandomForestClassifier,
)
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import NearestNeighbors

# ---------------------------------------
# PRD-driven feature engineering
# ---------------------------------------

def add_prd_logic_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds PRD-derived behavioral features:
      - spend_per_day curved behavior
      - efficiency reward band (4–6 days & 180–220 mi/day)
      - log miles decay
      - receipt cents anomalies (.49 / .99)
      - receipts curve near $700 (bonus)
      - extreme receipt conditions

    Assumes Phase 2 features already exist:
        cost_per_day, miles_traveled, trip_duration_days, total_receipts_amount
    """
    df = df.copy()

    # === Core Derived Behavior ===
    # Per-day spend: in Phase 2, cost_per_day already behaves like spend_per_day
    df["spend_per_day"] = df["cost_per_day"]

    # Miles per day with safe denominator
    df["miles_per_day_safe"] = (
        df["miles_traveled"] / df["trip_duration_days"].clip(lower=1)
    )

    # Log miles for non-linear mileage curve
    df["log_miles_traveled"] = np.log1p(df["miles_traveled"])

    # === Receipt cents anomalies (.49 / .99 quirk) ===
    df["receipt_cents"] = np.round(
        df["total_receipts_amount"] - np.floor(df["total_receipts_amount"]), 2
    )
    df["receipt_is_point_49_or_99"] = df["receipt_cents"].isin([0.49, 0.99]).astype(int)

    # === Spend per day optimal band (75–120) & penalties ===
    df["spend_per_day_good_band"] = df["spend_per_day"].between(75, 120).astype(int)
    df["spend_per_day_low"] = (df["spend_per_day"] < 50).astype(int)
    df["spend_per_day_high"] = (df["spend_per_day"] > 120).astype(int)

    # === Receipt non-linearity around ~700 ===
    df["receipts_near_700"] = df["total_receipts_amount"].between(600, 800).astype(int)
    df["receipts_very_low"] = (df["total_receipts_amount"] < 50).astype(int)
    df["receipts_very_high"] = (df["total_receipts_amount"] > 1000).astype(int)

    # === Efficiency Sweet Spot (4–6 days ∧ 180–220 mi/day) ===
    duration_mask = df["trip_duration_days"].between(4, 6)
    miles_mask = df["miles_per_day_safe"].between(180, 220)

    df["is_efficiency_sweet_spot"] = (duration_mask & miles_mask).astype(int)
    df["efficiency_score"] = (
        df["miles_per_day_safe"] * df["is_efficiency_sweet_spot"]
    )

    return df


# ----------------------------------------------------------
# Load Phase 2 dataset + add PRD features
# ----------------------------------------------------------
combined_df = pd.read_csv("phase2_features_baseline_models.csv")

print("Dataset loaded for Phase 3!")
print("Shape:", combined_df.shape)

combined_df = add_prd_logic_features(combined_df)
print("PRD-driven logic features added. New shape:", combined_df.shape)

# ---------------------------------------
# Feature Selection (ONLY original + PRD)
# ---------------------------------------
features = [
    # Phase 2 continuous engineered inputs
    "trip_duration_days",
    "miles_traveled",
    "total_receipts_amount",
    "cost_per_day",
    "cost_per_mile",
    "miles_per_day",
    "cost_ratio",

    # PRD logic features
    "spend_per_day",
    "miles_per_day_safe",
    "log_miles_traveled",
    "receipt_cents",
    "receipt_is_point_49_or_99",
    "spend_per_day_good_band",
    "spend_per_day_low",
    "spend_per_day_high",
    "receipts_near_700",
    "receipts_very_low",
    "receipts_very_high",
    "efficiency_score",
    "is_efficiency_sweet_spot",
]

target = "reimbursement"

X = combined_df[features]
y = combined_df[target]
# ==========================================================
# Sanity Checks + Random (Shuffled) Train/Test Split
# ==========================================================

from sklearn.model_selection import train_test_split

print("\n===== 🔍 Sanity Checks =====")

# 1) How quantized is reimbursement? (lots of repeated exact values suggests discrete payout bands)
y_rounded = combined_df[target].round(2)
n_unique = y_rounded.nunique()
top_counts = y_rounded.value_counts().head(15)

print(f"Unique reimbursements (rounded to cents): {n_unique}")
print("Most common payouts (top 15):")
print(top_counts.to_string())

# 2) Do identical feature rows map to multiple reimbursements? (hidden inputs/noise)
tmp = combined_df[features + [target]].copy()
tmp[target] = tmp[target].round(2)  # compare at cent-level

# group by X and count unique y values per identical X row
y_variation = tmp.groupby(features)[target].nunique()
multi_y_count = int((y_variation > 1).sum())
max_y_for_same_x = int(y_variation.max())

print(f"Identical X mapping to >1 reimbursement (cent-rounded): {multi_y_count}")
print(f"Max distinct reimbursements for the same X: {max_y_for_same_x}")

if multi_y_count > 0:
    print("⚠️  Warning: Same inputs sometimes produce different outputs.")
    print("    This suggests hidden variables and/or stochastic behavior, which limits match-rate ceiling.")

# ==========================================================
# Random / Shuffled split (recommended vs first-75% / last-25%)
# ==========================================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    shuffle=True
)

print("\n===== ✅ Random Split Applied =====")
print(f"Training Samples: {len(X_train)}")
print(f"Testing Samples:  {len(X_test)}")



# ----------------------------------------------------------
# Define Base & Stacking
# ----------------------------------------------------------
base_models = [
    ("decision_tree", DecisionTreeRegressor(random_state=42)),
    ("random_forest", RandomForestRegressor(n_estimators=200, random_state=42)),
    ("gradient_boosting", GradientBoostingRegressor(random_state=42)),
]

meta_model = LinearRegression()

stack_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True,
)

stack_model.fit(X_train, y_train)

# ----------------------------------------------------------
# Evaluate Base Stacked Model
# ----------------------------------------------------------
stack_pred = stack_model.predict(X_test)

mae = mean_absolute_error(y_test, stack_pred)
rmse = sqrt(mean_squared_error(y_test, stack_pred))
r2 = r2_score(y_test, stack_pred)

print("\n Ensemble Model Performance (Stacking Regressor):")
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")


# ----------------------------------------------------------
# Prediction Match Test helper
# ----------------------------------------------------------
def prediction_match_report(y_true, y_pred, label="Phase 3 Prediction Match Test"):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    abs_diff = np.abs(y_pred - y_true)
    print(f"\n=====  {label} =====")
    print(f"Exact Match Rate (<= $0.01): {(abs_diff <= 0.01).mean():.4f}")
    print(f"Close Match Rate (<= $1.00): {(abs_diff <= 1.00).mean():.4f}")
    print(f"±$5 Accuracy: {(abs_diff <= 5.00).mean():.4f}")


print("\n=====  Base Ensemble Match Rates (Test) =====")
prediction_match_report(y_test, stack_pred, label="Base Ensemble Match Rates (Test)")

# ----------------------------------------------------------
# Residual Calibration
# ----------------------------------------------------------
from sklearn.ensemble import GradientBoostingRegressor as GBRResidual

train_base_pred = stack_model.predict(X_train)
train_residuals = y_train - train_base_pred

residual_model = GBRResidual(
    random_state=42,
    n_estimators=100,
    learning_rate=0.05,
    max_depth=2,
    min_samples_leaf=20,
)

residual_model.fit(X_train, train_residuals)

print("\n Residual correction model trained!")

# Corrections
train_corrections = residual_model.predict(X_train)
test_corrections = residual_model.predict(X_test)

# Calibrated predictions
train_final_pred = train_base_pred + train_corrections
test_final_pred = stack_pred + test_corrections

# Calibrated metrics
calib_mae = mean_absolute_error(y_test, test_final_pred)
calib_rmse = sqrt(mean_squared_error(y_test, test_final_pred))
calib_r2 = r2_score(y_test, test_final_pred)

print("\n  Calibrated Ensemble Performance (with Residual Model):")
print(f"MAE:  {calib_mae:.4f}")
print(f"RMSE: {calib_rmse:.4f}")
print(f"R²:   {calib_r2:.4f}")

print("\n=====  Calibrated Ensemble Match Rates (Test – Raw) =====")
prediction_match_report(y_test, test_final_pred, label="Calibrated Ensemble Match Rates (Test – Raw)")

# Optional: rounded to cents
test_final_pred_rounded = np.round(test_final_pred, 2)
print("\n=====  Calibrated Ensemble Match Rates (Test – Rounded to Cents) =====")
prediction_match_report(
    y_test, test_final_pred_rounded, label="Calibrated Ensemble Match Rates (Test – Rounded to Cents)"
)

print("\n=====  Calibrated Ensemble Match Rates (Train) =====")
prediction_match_report(y_train, train_final_pred, label="Calibrated Ensemble Match Rates (Train)")


# ==========================================================
# Advanced Techniques – SVR, MLP, Voting Ensemble
# ==========================================================

# --- Support Vector Regression (SVR) ---
svr_model = make_pipeline(
    StandardScaler(),
    SVR(kernel="rbf", C=10.0, epsilon=5.0),
)

svr_model.fit(X_train, y_train)
svr_pred = svr_model.predict(X_test)

svr_mae = mean_absolute_error(y_test, svr_pred)
svr_rmse = sqrt(mean_squared_error(y_test, svr_pred))
svr_r2 = r2_score(y_test, svr_pred)

print("\n SVR Performance:")
print(f"MAE:  {svr_mae:.4f}")
print(f"RMSE: {svr_rmse:.4f}")
print(f"R²:   {svr_r2:.4f}")
prediction_match_report(y_test, svr_pred, label="SVR Match Rates (Test)")


# --- Neural Network (MLPRegressor) ---
mlp_model = make_pipeline(
    StandardScaler(),
    MLPRegressor(
        hidden_layer_sizes=(64, 32),
        activation="relu",
        solver="adam",
        alpha=1e-3,
        max_iter=2000,
        random_state=42,
    ),
)

mlp_model.fit(X_train, y_train)
mlp_pred = mlp_model.predict(X_test)

mlp_mae = mean_absolute_error(y_test, mlp_pred)
mlp_rmse = sqrt(mean_squared_error(y_test, mlp_pred))
mlp_r2 = r2_score(y_test, mlp_pred)

print("\n MLP Performance:")
print(f"MAE:  {mlp_mae:.4f}")
print(f"RMSE: {mlp_rmse:.4f}")
print(f"R²:   {mlp_r2:.4f}")
prediction_match_report(y_test, mlp_pred, label="MLP Match Rates (Test)")


# --- Voting Regressor (simple averaging ensemble) ---
voting_reg = VotingRegressor(
    estimators=[
        ("rf", base_models[1][1]),   # RandomForestRegressor
        ("gbr", base_models[2][1]),  # GradientBoostingRegressor
        ("tree", base_models[0][1]), # DecisionTreeRegressor
    ]
)

voting_reg.fit(X_train, y_train)
voting_pred = voting_reg.predict(X_test)

v_mae = mean_absolute_error(y_test, voting_pred)
v_rmse = sqrt(mean_squared_error(y_test, voting_pred))
v_r2 = r2_score(y_test, voting_pred)

print("\n Voting Regressor Performance:")
print(f"MAE:  {v_mae:.4f}")
print(f"RMSE: {v_rmse:.4f}")
print(f"R²:   {v_r2:.4f}")
prediction_match_report(y_test, voting_pred, label="Voting Regressor Match Rates (Test)")


# ==========================================================
# Rule-Based Learning – Surrogate Tree for Interpretability
# ==========================================================

# Train a shallow tree to mimic the calibrated ensemble behavior
surrogate_tree = DecisionTreeRegressor(
    max_depth=4,
    min_samples_leaf=20,
    random_state=42,
)

# Use calibrated train predictions as the "target" for the surrogate
surrogate_tree.fit(X_train, train_final_pred)

tree_rules = export_text(surrogate_tree, feature_names=list(X.columns))

print("\n=== Surrogate Tree Rules (approximate learned logic) ===")
print(tree_rules)


# ==========================================================
# DISCRETE / TIERED MODELING – Classification over Payout Buckets
# (FIXED for random train_test_split indexing)
# ==========================================================

from sklearn.neighbors import KNeighborsRegressor

bucket_size = 5.0  # try 5.0, 2.0, 1.0 (but 1.0 creates many classes)

# Build bucket labels from y (NOT from sequential slicing)
y_bucket_all = (y.round(2) / bucket_size).round().astype(int)

# Align buckets with the randomized split via indices
y_bucket_train = y_bucket_all.loc[X_train.index]
y_bucket_test  = y_bucket_all.loc[X_test.index]

# Map: bucket -> representative payout (median payout in TRAIN for that bucket)
train_bucket_df = pd.DataFrame({
    "bucket": y_bucket_train,
    "payout": y_train
})
bucket_to_median = train_bucket_df.groupby("bucket")["payout"].median().to_dict()

def buckets_to_payouts(bucket_array):
    bucket_array = np.asarray(bucket_array)
    return np.array([bucket_to_median.get(int(b), float(b) * bucket_size) for b in bucket_array], dtype=float)

rf_cls = RandomForestClassifier(
    n_estimators=400,
    max_depth=None,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)
rf_cls.fit(X_train, y_bucket_train)

bucket_pred_test = rf_cls.predict(X_test)
tier_pred_test = buckets_to_payouts(bucket_pred_test)

print("\n Tier Classifier (RF over buckets) Performance:")
print(f"MAE:  {mean_absolute_error(y_test, tier_pred_test):.4f}")
print(f"RMSE: {sqrt(mean_squared_error(y_test, tier_pred_test)):.4f}")
print(f"R²:   {r2_score(y_test, tier_pred_test):.4f}")
prediction_match_report(y_test, tier_pred_test, label="Tier Classifier Match Rates (Test)")


# ==========================================================
# CASE-BASED REASONING – kNN Regressor (IMPORTANT: scaling + distance weights)
# ==========================================================

# Your previous NearestNeighbors version used raw feature scales
# => distances dominated by receipts/miles. Scale helps a lot for neighbor quality.

knn_reg = make_pipeline(
    StandardScaler(),
    KNeighborsRegressor(
        n_neighbors=3,         # try 1,3,5,7
        weights="distance",    # closer neighbors matter more
        metric="minkowski",
        p=2
    )
)

knn_reg.fit(X_train, y_train)
knn_pred = knn_reg.predict(X_test)

print("\n kNN Regressor Performance (scaled, distance-weighted):")
print(f"MAE:  {mean_absolute_error(y_test, knn_pred):.4f}")
print(f"RMSE: {sqrt(mean_squared_error(y_test, knn_pred)):.4f}")
print(f"R²:   {r2_score(y_test, knn_pred):.4f}")
prediction_match_report(y_test, knn_pred, label="kNN Regressor Match Rates (Test)")



